Import required packages

In [45]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import FunctionTransformer
from sklearn import metrics
from sklearn.metrics import accuracy_score, mean_squared_error
from sklearn.feature_selection import SelectKBest, SelectPercentile
from sklearn.feature_selection import chi2, f_regression, mutual_info_classif
import pandas as pd
import numpy as np

Load in dataset, set column headers

In [112]:
# load in training/test set
data = pd.read_csv('chicago.csv', encoding='utf-8', delimiter=',')
data.head()

,doc_id,hotel_name,hotel_url,street,city,state,country,zip,class,price,num_reviews,CLEANLINESS,ROOM,SERVICE,LOCATION,VALUE,COMFORT,overall_ratingsource
0,usa_illinois_chicago_the_talbott_hotel,the talbott hotel,http://www.tripadvisor.com/ShowUserReviews-g35...,-1,chicago,il,usa,60611,-1,-1,212,4.729508,4.655738,4.647541,4.844262,4.483607,0,4.672131
1,usa_illinois_chicago_sofitel_chicago_water_tower,sofitel chicago water tower,http://www.tripadvisor.com/ShowUserReviews-g35...,-1,chicago,il,usa,60611,-1,-1,346,4.760797,4.558140,4.604651,4.794020,4.289037,0,4.601329
2,usa_illinois_chicago_trump_international_hotel...,trump international hotel tower chicago,http://www.tripadvisor.com/ShowUserReviews-g35...,-1,chicago,il,usa,60611,-1,-1,128,4.888889,4.842593,4.574074,4.861111,4.314815,0,4.696296
3,usa_illinois_chicago_hampton_inn_majestic_chicago,hampton inn majestic chicago,http://www.tripadvisor.com/ShowUserReviews-g35...,-1,chicago,il,usa,60603,-1,-1,141,4.757576,4.590909,4.492424,4.742424,4.393939,0,4.595455
4,usa_illinois_chicago_residence_inn_chicago_dow...,residence inn chicago downtown river north,http://www.tripadvisor.com/ShowUserReviews-g35...,-1,chicago,il,usa,60610,-1,-1,82,4.787500,4.600000,4.650000,4.725000,4.462500,0,4.645000


In [113]:
import os.path
ratings = data['overall_ratingsource'].tolist()
docids = data['doc_id'].tolist()
reviews = []
for docid in docids:
    if os.path.exists('chicago/'+str(docid)):
        
        f = open('chicago/'+str(docid),'rb')
        review = f.readlines()
        reviews.append(review[0].decode('utf-8', 'ignore'))
    else:
        reviews.append("placeholder")
data = pd.DataFrame({'docid':docids, 'review':reviews, 'rating':ratings})
data.to_csv('chicago_hotel_reviews.csv',header=True)
data['docid']

0                 usa_illinois_chicago_the_talbott_hotel
1       usa_illinois_chicago_sofitel_chicago_water_tower
2      usa_illinois_chicago_trump_international_hotel...
3      usa_illinois_chicago_hampton_inn_majestic_chicago
4      usa_illinois_chicago_residence_inn_chicago_dow...
5      usa_illinois_chicago_hotel_monaco_chicago_a_ki...
6                usa_illinois_chicago_park_hyatt_chicago
7      usa_illinois_chicago_springhill_suites_chicago...
8      usa_illinois_chicago_homewood_suites_by_hilton...
9                   usa_illinois_chicago_affinia_chicago
10                   usa_illinois_chicago_dana_hotel_spa
11            usa_illinois_chicago_the_peninsula_chicago
12     usa_illinois_chicago_the_blackstone_a_renaissa...
13                    usa_illinois_chicago_james_chicago
14             usa_illinois_chicago_amalfi_hotel_chicago
15       usa_illinois_chicago_four_seasons_hotel_chicago
16     usa_illinois_chicago_hotel_burnham_a_kimpton_h...
17                   usa_illino

In [4]:
#colnames = ['label','id','date','query','user','text']
#data_full = pd.read_csv('training.1600000.processed.noemoticon.csv', header=None, names=colnames, encoding='windows-1252')
#data_full = data_full.sample(n=160000)
#data_full.to_csv('tweets.160k.random.csv', sep=',', encoding='utf-8', index=False, index_label=True)

**The solution starts here. Define the various components of our pipeline

In [114]:
# build a pipeline components for uni-grams and bi-grams, using a 80:20 train/test split
data = pd.read_csv('chicago_hotel_reviews.csv', encoding='utf-8', delimiter=',')
labels = data['rating']
x_train, x_test, y_train, y_test = train_test_split(data, labels, test_size=0.2)
getTweetCol = FunctionTransformer(lambda x: x['review'], validate=False) # extract reviews
tfVect = CountVectorizer(stop_words='english', lowercase=True, ngram_range=(1,2))
linReg = LinearRegression()

Build pipeline for Linear Regression regressor using unigrams and bigrams

In [117]:
clf_tf = Pipeline([('text', getTweetCol), ('vect', tfVect), ('clf', linReg)])
clf_tf.fit(x_train, y_train)
predicted = clf_tf.predict(x_test)
print(mean_squared_error(y_test, predicted))

0.8170495362493827


Build similar pipeline, with feature selection

In [116]:
feaSelect = SelectPercentile(f_regression, percentile=8)
clf_tf = Pipeline([('text', getTweetCol), ('vect', tfVect), 
                   ('feaSelect', feaSelect), ('clf', linReg)])
clf_tf.fit(x_train, y_train)
predicted = clf_tf.predict(x_test)
print(mean_squared_error(y_test, predicted))

2.1035555387011335


/anaconda3/lib/python3.6/site-packages/sklearn/feature_selection/univariate_selection.py:114: RuntimeWarning: divide by zero encountered in true_divide
  f = msb / msw
